In [22]:
import sys
sys.path.insert(0, "../shared_sources")
import bird_dataset as loader

from stft import CalcMel
from sklearn.preprocessing import MaxAbsScaler
from ipywidgets import IntProgress
from IPython.display import display
import numpy as np
import pickle

### load mel_spectogram calculator. the transformer will be used for scaling durig the training of the wgan. For now it just passes the values

In [15]:
class transformer():
    def transform(self, y):
        return y
# transformer() will be replaced with the trained maxabscaler during training of the wgan
calc_mel = CalcMel(transformer(), (40,1,80,80))

### setup dataloader witch creates batches of mel-spectograms for training

In [16]:
data_path = "/media/fabi/terra/birdsongs/data/wavs/"
class_folders = ["Willow Warbler ", "Great Tit "]

slice_length = 0.92 # seconds
desired_semplerate = 22050
debug = False
rewrite_npy = False

generator = loader.CustomDataGenerator(data_path, class_folders, slice_length, desired_semplerate, 
                                 1, 40, -1, calc_mel.mel_spectrogram,1, debug, rewrite_npy)

train_generator,_  = generator.get_generators()
print(len(train_generator))

2616


### init maxabscaler

In [17]:
scaler = MaxAbsScaler()
scalerfile = 'fitted_maxabsscalerx80.sav'

### train maxabscaler

In [ ]:
i = 0
f = IntProgress(min=0, max=len(train_generator))
display(f)

for batch,_ in train_generator:
    f.value += 1
    for im in batch:
        im = np.reshape(im.numpy(),(80,80))
        scaler.partial_fit(im)
    if i % 1000 == 0:
        pickle.dump(scaler, open(scalerfile, 'wb'))
    i = i +1

pickle.dump(scaler, open(scalerfile, 'wb'))